# Decision Trees

Decision trees are one of the most powerful and popular prediction methods used nowadays. They are used in many fields and are the basis of more powerful techniques such as __Bagging__ and __Boosting__.

One of the biggest selling points of decision trees is that the final model can be understood by experts and novices because, in the end, it consists of a series of rules that comprise a decision graph and, hence, each decision can be tracked through the tree.

Although decision trees can be used in regression and classification problems, in this notebook we'll focus on the latter case. For that matter we'll compute something called __Gini index__ or __Gini cost function__ which measures the purity of a particular node in the tree. A node is 100% pure if all the rows or examples that comprise it are of the same class.

Let's start our implementation by loading the code and libraries we'll need. We will build our solution on top of the ones we implemented in the [previous notebook](https://github.com/jesus-a-martinez-v/toy-ml/blob/master/src/main/scala/notebooks/linear_vector_quantization.ipynb).

In [1]:
import $ivy.`com.github.tototoshi::scala-csv:1.3.5`
import $file.^.datasmarts.ml.toy.scripts.LinearVectorQuantization, LinearVectorQuantization._
import scala.util.Random

Compiling LinearVectorQuantization.sc


import $ivy.$                                      

import $file.$                                                   , LinearVectorQuantization._

import scala.util.Random

## Data

We'll use the [Banknote](http://archive.ics.uci.edu/ml/machine-learning-databases/00267/data_banknote_authentication.txt) dataset. It involves the prediction of the authenticity of a banknote based on a given number of measures extracted from a photograph. It is a binary classification task.

Let's load the data:

In [2]:
val BASE_DATA_PATH = "../../resources/data"
val banknotePath = s"$BASE_DATA_PATH/11/banknote.csv"

val rawData = loadCsv(banknotePath)
val numberOfRows = rawData.length
val numberOfColumns = rawData.head.length
println(s"Number of rows in dataset: $numberOfRows")
println(s"Number of columns in dataset: $numberOfColumns")

val (data, lookUpTable) = {
    val dataWithNumericColumns = (0 until (numberOfColumns - 1)).toVector.foldLeft(rawData) { (d, i) => textColumnToNumeric(d, i) }
    categoricalColumnToNumeric(dataWithNumericColumns, numberOfColumns - 1)
}

Number of rows in dataset: 1372
Number of columns in dataset: 5


BASE_DATA_PATH: String = "../../resources/data"
banknotePath: String = "../../resources/data/11/banknote.csv"
rawData: Vector[Vector[Data]] = Vector(
  Vector(Text(3.6216), Text(8.6661), Text(-2.8073), Text(-0.44699), Text(0)),
  Vector(Text(4.5459), Text(8.1674), Text(-2.4586), Text(-1.4621), Text(0)),
  Vector(Text(3.866), Text(-2.6383), Text(1.9242), Text(0.10645), Text(0)),
  Vector(Text(3.4566), Text(9.5228), Text(-4.0112), Text(-3.5944), Text(0)),
  Vector(Text(0.32924), Text(-4.4552), Text(4.5718), Text(-0.9888), Text(0)),
  Vector(Text(4.3684), Text(9.6718), Text(-3.9606), Text(-3.1625), Text(0)),
  Vector(Text(3.5912), Text(3.0129), Text(0.72888), Text(0.56421), Text(0)),
  Vector(Text(2.0922), Text(-6.81), Text(8.4636), Text(-0.60216), Text(0)),
  Vector(Text(3.2032), Text(5.7588), Text(-0.75345), Text(-0.61251), Text(0)),
  Vector(Text(1.5356), Text(9.1772), Text(-2.2718), Text(-0.73535), Text(0)),
  Vector(Text(1.2247), Text(8.7779), Text(-2.2135), Text(-0.80647), Text(0)),

## Gini Index

Gini index is the cost function we'll use to determine how good is a split point in the dataset. 

A split involves a feature and a value for that feature. Then they are used to divide training patterns and nuances into two groups of rows. For instance, if we select feature _X1_ and value _0.5_, then we could have two groups of rows corresponding to __rows where X1 >= 0.5__ and __rows where X1 < 0.5__.

Then, the Gini score would be 0.0 if we have the perfect separation among classes and 1.0 if that split left us with the worst separation possible, which is 50/50 presence of each class in each group.

Let's implement a Gini index function:

In [3]:
type Group = Vector[Vector[Data]]  // Just a vector of rows

def giniIndex(groups: Vector[Group], classValues: Vector[Numeric]) = {
  classValues.foldLeft(0.0) { (accumulatedGini, classValue) =>
    groups.foldLeft(accumulatedGini) { (g, group) =>
      if (group.nonEmpty) {
        val proportion = group.map(_.last).count(_ == classValue).toDouble / group.length.toDouble
        g + proportion * (1.0 - proportion)
      } else {
        g
      }
    }
  }
}

defined type Group
defined function giniIndex

## Creating a Split

The first thing we need to do is actually creating a split. Let's start by implementing a function for it:

In [4]:
def testSplit(index: Int, value: Numeric, dataset: Dataset): Vector[Vector[Vector[Data]]] = {
  val numericValue = value.value
  val (left, right) = dataset.partition(r => getNumericValue(r(index)).get < numericValue)

  Vector(left, right)
}

defined function testSplit

Here we create two groups based on those rows where the feature is lesser than a specified value and those rows where the feature is greater than or equal to the provided value.

Now we need a way of selecting the best split, using our Gini index function.

In [5]:
sealed trait TreeNode
case class InnerNode(index: Int, value: Double, groups: Option[Vector[Group]] = None, left: Option[TreeNode] = None, right: Option[TreeNode] = None) extends TreeNode
case class TerminalNode(outcome: Numeric) extends TreeNode

def getSplit(dataset: Dataset) = {
  val classValues = selectColumn(dataset, dataset.head.length - 1).distinct
  
  var bIndex = 999
  var bValue = 999.0
  var bScore = 999.0
  var bGroups = Vector.empty[Group]

  for (index <- 0 until (dataset.head.length - 1)) {
    for (row <- dataset) {
      val groups = testSplit(index, row(index).asInstanceOf[Numeric], dataset)
      val gini = giniIndex(groups, classValues.asInstanceOf[Vector[Numeric]])

      if (gini < bScore) {
        bIndex = index
        bValue = row(index).asInstanceOf[Numeric].value
        bScore = gini
        bGroups = groups
      }
    }
  }

  InnerNode(index = bIndex, value = bValue, groups = Some(bGroups))
}

defined trait TreeNode
defined class InnerNode
defined class TerminalNode
defined function getSplit

Here we return the best split as an inner node of the tree. Basically, we return the index corresponding to the feature column in the dataset, the splitting value and the rows or group that comprise this split node.

Let's test this on a mock dataset:

In [6]:
val mockDataset = Vector(
  (2.771244718,1.784783929,0),
  (1.728571309,1.169761413,0),
  (3.678319846,2.81281357,0),
  (3.961043357,2.61995032,0),
  (2.999208922,2.209014212,0),
  (7.497545867,3.162953546,1),
  (9.00220326,3.339047188,1),
  (7.444542326,0.476683375,1),
  (10.12493903,3.234550982,1),
  (6.642287351,3.319983761,1)) map { case (x1, x2, y) => Vector(Numeric(x1), Numeric(x2), Numeric(y)) }

val split = getSplit(mockDataset)
println(split.index)
println(split.value)

0
6.642287351


mockDataset: Vector[Vector[Numeric]] = Vector(
  Vector(Numeric(2.771244718), Numeric(1.784783929), Numeric(0.0)),
  Vector(Numeric(1.728571309), Numeric(1.169761413), Numeric(0.0)),
  Vector(Numeric(3.678319846), Numeric(2.81281357), Numeric(0.0)),
  Vector(Numeric(3.961043357), Numeric(2.61995032), Numeric(0.0)),
  Vector(Numeric(2.999208922), Numeric(2.209014212), Numeric(0.0)),
  Vector(Numeric(7.497545867), Numeric(3.162953546), Numeric(1.0)),
  Vector(Numeric(9.00220326), Numeric(3.339047188), Numeric(1.0)),
  Vector(Numeric(7.444542326), Numeric(0.476683375), Numeric(1.0)),
  Vector(Numeric(10.12493903), Numeric(3.234550982), Numeric(1.0)),
  Vector(Numeric(6.642287351), Numeric(3.319983761), Numeric(1.0))
)
split: InnerNode = InnerNode(0,6.642287351,Some(Vector(Vector(Vector(Numeric(2.771244718), Numeric(1.784783929), Numeric(0.0)), Vector(Numeric(1.728571309), Numeric(1.169761413), Numeric(0.0)), Vector(Numeric(3.678319846), Numeric(2.81281357), Numeric(0.0)), Vector(Numeric(3

Good. What we see here is that our function decided that the best split for this dataset is on the feature at column 0, on the value 6.642287351.

## Building a Tree

We've arrived to the hardest part: building the actual tree.

A tree is comprised of __terminal nodes__ that are those nodes where we stop expanding based on a preconfigured maximum depth of the tree and the minimum number of rows in a group of a node. Terminal nodes are used to make final predictions. Given that they do not need to know about groups, or subtrees, we will only store on them the outcome (class label).

Let's create a function to instantiate a terminal node based on a group of rows:

In [7]:
def toTerminal(group: Group): TerminalNode = {
  val outcomes = group.map(_.last)
  TerminalNode {
    outcomes.maxBy(o => outcomes.count(_ == o)).asInstanceOf[Numeric]
  }
}

defined function toTerminal

Here the outcome is defined as the mode among the labels or classes.

Now, for the inner nodes of the tree we must apply a recursive process. Here's the outline of the function that'll do that:

  1. We extract the two groups of data split by the node.
  2. Then, if either the left or right group is empty, we'll proceed to create a terminal node using the rows we have.
  3. If we reached our maximum depth, we create a terminal node.
  4. Then, we process the left child creating a terminal node if the group of rows is less than _minimum size_. If not, we make a recursive call on the left group and the depth is incremented by one. Next, we process the right child in the same fashion.

In [8]:
def split(node: TreeNode, maxDepth: Int, minSize: Int, depth: Int): TreeNode = {
  node match {
    case terminalNode: TerminalNode =>
      terminalNode

    case innerNode @ InnerNode(index, value, Some(Vector(leftGroup, rightGroup)), _, _)
      if leftGroup.isEmpty || rightGroup.isEmpty =>

      val t = toTerminal(leftGroup ++ rightGroup)
      innerNode.copy(left = Some(t), right = Some(t))

    case innerNode: InnerNode if innerNode.groups.isDefined =>
      val groups: Vector[Group] = innerNode.groups.get
      val Vector(left, right) = groups

      if (depth >= maxDepth) {
        innerNode.copy(left = Some(toTerminal(left)), right = Some(toTerminal(right)))
      } else {
        val leftNode = if (left.lengthCompare(minSize) <= 0) {
            innerNode.copy(left = Some(toTerminal(left)))
        } else {
          val n = innerNode.copy(left = Some(getSplit(left)))
          split(n, maxDepth, minSize, depth + 1)
        }

        val rightNode = if (right.lengthCompare(minSize) <= 0) {
          innerNode.copy(right = Some(toTerminal(right)))
        } else {
          val n = innerNode.copy(right = Some(getSplit(right)))
          split(n, maxDepth, minSize, depth + 1)
        }

        innerNode.copy(left = Some(leftNode), right = Some(rightNode))
      }
  }
}

defined function split

## Building a Tree

Building a tree is now only a matter of calling `split` on the whole dataset, and specifying the desired maximum depth and minimum node size.

In [9]:
def buildTree(train: Dataset, maxDepth: Int, minSize: Int) = split(getSplit(train), maxDepth, minSize, 1)

defined function buildTree

Let's build a little tree using our mock dataset.

In [10]:
buildTree(mockDataset, 1, 1)

res9: TreeNode = InnerNode(0,6.642287351,Some(Vector(Vector(Vector(Numeric(2.771244718), Numeric(1.784783929), Numeric(0.0)), Vector(Numeric(1.728571309), Numeric(1.169761413), Numeric(0.0)), Vector(Numeric(3.678319846), Numeric(2.81281357), Numeric(0.0)), Vector(Numeric(3.961043357), Numeric(2.61995032), Numeric(0.0)), Vector(Numeric(2.999208922), Numeric(2.209014212), Numeric(0.0))), Vector(Vector(Numeric(7.497545867), Numeric(3.162953546), Numeric(1.0)), Vector(Numeric(9.00220326), Numeric(3.339047188), Numeric(1.0)), Vector(Numeric(7.444542326), Numeric(0.476683375), Numeric(1.0)), Vector(Numeric(10.12493903), Numeric(3.234550982), Numeric(1.0)), Vector(Numeric(6.642287351), Numeric(3.319983761), Numeric(1.0))))),Some(TerminalNode(Numeric(0.0))),Some(TerminalNode(Numeric(1.0))))

## Make Predictions

Making predictions is just the process of traversing the tree, selecting the branches that the different split nodes indicate until we reach a terminal node. Then, the prediction will be the outcome stored in that node.

In [11]:
def predictWithTree(node: TreeNode, row: Vector[Numeric]): Numeric = node match {
  case TerminalNode(outcome) => outcome
  case i: InnerNode =>
    if (getNumericValue(row(i.index)).get <= i.value) {
      predictWithTree(i.left.get, row)
    } else {
      predictWithTree(i.right.get, row)
    }
}

defined function predictWithTree

In [14]:
def decisionTree(train: Dataset, test: Dataset, parameters: Parameters) = {
  val maxDepth = parameters("maxDepth").asInstanceOf[Int]
  val minSize = parameters("minSize").asInstanceOf[Int]

  val t = buildTree(train, maxDepth, minSize)

  test.map { row =>
    predictWithTree(t, row.asInstanceOf[Vector[Numeric]])
  }
}

defined function decisionTree

Let's make prediction on our mock dataset.

In [13]:
for (r <- mockDataset) {
  println(s"Expected=${r.last.value}, Got=${predictWithTree(buildTree(mockDataset, 1, 1), r).value}")
}

Expected=0.0, Got=0.0
Expected=0.0, Got=0.0
Expected=0.0, Got=0.0
Expected=0.0, Got=0.0
Expected=0.0, Got=0.0
Expected=1.0, Got=1.0
Expected=1.0, Got=1.0
Expected=1.0, Got=1.0
Expected=1.0, Got=1.0
Expected=1.0, Got=0.0


Good.

Let's now use our new algorithm to test it on the Banknote dataset.

We'll start by running a baseline model on it and then our freshly implemented Decision Tree algorithm and then we will compare their performance.

As a baseline for classification we will use a __zero rule classifier__.

In [15]:
val baselineAccuracy = evaluateAlgorithmUsingTrainTestSplit[Numeric](
  data,
  (train, test, parameters) => zeroRuleClassifier(train, test),
  Map.empty,
  accuracy,
  trainProportion=0.8)

println(s"Zero Rule Algorithm accuracy: $baselineAccuracy")

Zero Rule Algorithm accuracy: 0.5927272727272728


baselineAccuracy: Double = 0.5927272727272728

In [16]:
val decisionTreeAccuracy = evaluateAlgorithmUsingTrainTestSplit[Numeric](
  data,
  decisionTree,
  Map("maxDepth" -> 5, "minSize" -> 10),
  accuracy,
  trainProportion=0.8)

println(s"Decision Tree accuracy: $decisionTreeAccuracy")

Decision Tree accuracy: 0.7818181818181819


decisionTreeAccuracy: Double = 0.7818181818181819

Great. As expected, our __decision tree__ performs a lot better than our baseline. 

Of course, playing around with different depths and node sizes could potentially yield a better score, so I encourage you to play and beat my accuracy!